### Занятие 8. АБ-тестирование (проверяем все в онлайн-экспериментах)

### План занятия

1. вспоминаем самое первое занятие
2. "типичное" устройство АБ-тестов
3. метрики для сравнения
4. оценка размера выборки ДО проведения эксперимента
5. параметрические и непараметрические критерии

Пример как может выглядеть решение задачи бизнеса с помощью ML

<img src="tdsp-lifecycle2.png" alt="TDSP" style="width: 800px;" align="center"/>

Если же сделать процесс более "линейным", то он может выглядеть так:

Проблема -> Анализ -> Продуктовая гипотеза -> Метрики и критерии приемки -> ML решение -> AB-тестирование -> Оценка результатов -> Интеграция с другими командами

#### Что такое "продуктовая гипотеза"?

Предположение о том, что мы ожидаем при том или ином изменении в продукте (новый функционал вроде системы рекомендаций или улучшение уже существующего). Под изменением обычно подразумевается улучшение некоторой бизнес-метрики. 

### Пример - сервис для чтения книг онлайн

- чтение книг онлайн
- монетизация в основном за счет подписок + платный контент
- стабильный DAU**
<img src="books.png" alt="reading" style="width: 400px;" align="right"/>

** DAU (daily active users) - число уникальных пользователей за сутки

Что же мы хотим сделать? - улучшить наш сервис, чтобы он стал более удобным для пользователей и более прибыльным для бизнеса.

Один из вариантов это сделать - <b>персональные рекомендации</b>**

** по персональным рекомендациям и как такие системы строятся - есть курс на geekbrains

Предположим, что мы со своей частью справляемся и можем сделать простейшую систему рекомендаций книг.

Что дальше?

## Сразу в "прод"?
<img src="good_enough.jpg" alt="good_enough" style="width: 200px;" align="center"/>

## Конечно же нет!

#### Что нам нужно сделать?

1. Формулировка задачи и выбор метрики (DS)
2. Сбор данных и подготовка датасета
3. Построение модели (обучение)
4. Оборачивание всего этого в пайплайн обучения (для того, чтобы использовать в production)
5. Оценка результатов и проведение экспериментов
6. Мониторинг качества, логгирование

### Практический пример 1

Допустим, что мы занимаемся рекомендациями в интернет-магазине.<br>
У нас уже есть рекомендательная система на основе метода A. 

При этом в результате исследований метод B показал лучшие по сравнению с А офлайн-метрики (обычно ML-метрики)

Означает ли, что такое решение будет лучше и с точки зрения бизнес-метрик? - Необязательно, мы должны будем это проверить. 

#### Вопрос 1
А что если альтернативных вариантов очень много - как выбирать? 

#### Вопрос 2
Почему мы не можем тестировать их все? Или можем?

### Метрика для нашего примера?

В случае с рекомендательной системой может быть много разных метрик, которые нам в итоге будут интересны:

1. arpu (average revenue per user) - средняя выручка одного пользователя
2. appu (average purchases per user) - среднее количество покупок на пользователя
3. crp (conversion rate payment) - доля пользователей, совершивших платеж
4. retention_N_day - удержание N-го дня
5. etc

*не всегда эти метрики связаны с revenue (например, количество кликов по баннеру или количество кликов по рекомендуемому фильму/книге/песне и т.д)

#### Вопрос 3
Можно ли измерять сразу несколько метрик?

#### Вопрос 4
Какие вы можете придумать "комбинированные" метрики?

### АБ тестирование

Предположим что с метрикой мы определились (вообще - на практике обычно измеряют сразу несколько метрик).

Далее мы собрали данные и обучили новую рекомендательную модель. Настало время проверить ее на практике и сравнить по выбранным метрикам с текущим решением!

Как может быть устроен АБ-тест:

![ab_split](ab1.png "AB split")

Конечно же групп может быть гораздо больше (если позволяет размер аудитории и дизайн эксперимента), но такой случай не только самый простой в плане проведения, но и оценки результатов. 

#### Вопрос 6

Каким должно быть соотношение размеров выборок? (спойлер - вопрос нетривиальный)

Итак, предположим, что мы запустили АБ тест и подождали, когда накопится статистика

Загрузим данные

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd

In [2]:
data = pd.read_csv("ab_stats.csv")
data.head(3)

,revenue,num_purchases,purchase,ab_group
0,0.0,0.0,0,A
1,0.0,0.0,0,B
2,0.0,0.0,0,A


Посмотрим на группы отдельно

In [3]:
data[data['ab_group']=='A'].describe()

,revenue,num_purchases,purchase
count,11835.000000,11835.000000,11835.000000
mean,0.404462,0.050697,0.021631
std,13.133218,1.467511,0.145481
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000
max,1303.609284,152.000000,1.000000


Группа B

In [4]:
data[data['ab_group']=='B'].describe()

,revenue,num_purchases,purchase
count,11817.000000,11817.000000,11817.000000
mean,0.244794,0.036473,0.019802
std,3.176534,0.418480,0.139325
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000
max,113.830000,25.000000,1.000000


Пока мы еще ничего не посчитали, но уже можно заметить, что максимальный чек в первой группе сильно больше, чем в группе B. Все мы знаем, что среднее очень неустойчиво к выбросам, так что нам необходимо будет это учесть.

Итак, начнем с того, что рассмотрим, какие вообще бывают статистические критерии

![ab_split](classification-non-parametric-1.png "AB parametric")

Если мы делаем предположение о нормальности распределения выбранной величины, то мы выбираем параметрические критерии - они как правило в такой ситуации являются более мощными.

Если же мы не может делать такое предположение, то наш выбор - непараметрические критерии.

Параметрические тесты основаны на нормальном распределении. Вспомним как оно выглядит:

<img src="normal_distr.png" alt="Normal distribution" style="width: 560px;" align="center"/>

Нормальное распределение:

* унимодально
* симметрично
* отклонения от среднего подчиняются конкретному вероятностному закону

Например: 
1. в диапазоне +- 1sigma* у нас лежит примерно 68.3% наблюдений 
2. в диапазоне +- 2sigma у нас лежит примерно 95.4% наблюдений

*sigma - стандартное отклонение

<b>Почему для нас это важно?</b> - вспомним центральную предельную теорему

<b>Пример задачи</b>

Вам дают монетку и просят оценить, насколько она "честная". Т.е вас просят сравнить значение вероятности выпадения орла с теоретическим значением 0.5.

По центральной предельной теореме (https://en.wikipedia.org/wiki/Central_limit_theorem), если мы будем семплировать и считать средние, то распределение этих средних будет <b>нормальным</b> и позволит нам оценить истинное значение некоторого параметра в генеральной совокупности. 

![CLT](CPT.png "Central limit theorem")

Например, мы пытаемся оценить значение некоторого параметра по имеющейся выборке (обозначим как X). Допустим, что это вероятность выпадения орла в серии бросков одной и той же монетки (обозначим данный параметр как p). У нас схема Бернулли, зависящая от параметра p (он нам неизвестен).

![bernouli](bernouli.png "bernouli")

Мы проводим серию бросков, а затем начинаем семлировать из полученной выборки и считать долю выпавших орлов на каждом шаге (k раз). Получаем уже последовательность из k выборочных значений вероятности выпадения орла (выборочные средние). Далее строим распределение из полученных значений и оно, по мере увеличения семплов, будет все больше похоже на нормальное. 

При этом параметры такого распределения будут mu и <b>s</b>, где mu - истинное значение параметра в генеральной совокупности, а s называют стандартной ошибкой среднего (<b>SE</b> - Standard Error).

![cpt1](cpt1.png "CPT")

Здесь s - это стандартное отклонение нашей выборки, n - размер выборки

s рассчитывается так, потому что у нас распределение Бернулли и формулы для него такие:

![bernouli2](bernouli2.png "bernouli2")

Математическое ожидание такого распределения равно выборочному p, а стандартное отклонение SE.

Получаем:
1. нормальное распределение
2. параметры известны (математическое ожидание и дисперсия)
3. можем рассчитать доверительные интервалы и делать выводы

Критериев вообще существует великое множество, но выбор конкретного критерия будет также зависеть и от того, какая у вас метрика:

источник - http://www.market-journal.com/marketingovyeissledovanija/161.html

<img src="parametric_nonparametric.jpg" alt="AB parametric/nonparametric" style="width: 600px;" align="center"/>

Итак, давайте рассмотрем на нашем примере

In [5]:
data.head(3)

,revenue,num_purchases,purchase,ab_group
0,0.0,0.0,0,A
1,0.0,0.0,0,B
2,0.0,0.0,0,A


In [6]:
data['purchase'].value_counts()

0    23162
1      490
Name: purchase, dtype: int64

Столбец purchase - это как раз таки бинарный показатель, является ли пользователем платящим или нет.

Наиболее очевидным статистическим критерием для нашего случая выглядит z-критерий.

Давайте здесь сделаем шаг назад и поймем, а что нам вообще может "рассказать" статистический критерий (тест)

#### Как обычно проходит тест:

1. случайно разбиваем аудиторию на две группы: A (обычно ее называют "контролем") и B (группа "экспериментальная")
2. группа А - текущий вариант модели в проде. Группа B - наша новая модель
3. по окончанию эксперимента мы сравниваем бизнес-метрику (вероятно даже не одну) в каждой из групп с помощью <b>статистического теста</b>

У нас 2 гипотезы:
- "нулевая" (H0) - наблюдаемые различия незначимы
- "альтернативная" (H1) - различия значимы

Мы хотим получить вероятность того, что наблюдаемые различия неслучайны!

#### Вопрос 7
Будет ли сам факт различия двух метрик (конверсий, к примеру) говорить нам о том, что варианты отличаются?

#### Вопрос 8

Если вы получаете p_value, равный 0.01 - это много или мало?

вероятность получить для данной вероятностной модели распределения значений случайной величины такое же или более экстремальное значение статистики (среднего арифметического, медианы и др.), по сравнению с ранее наблюдаемым, при условии, что нулевая гипотеза верна.


#### Теперь к нашей первой метрике - конверсии в платящего (у нас это столбец purchase)

Т.к у нас метрика - доля (конверсия), то применяем z-критерий

![z_score](z_score.png "Z score")

Гипотезы

![z_score_H01](z_score_H01.png "Z score H0/H1")

Применим z-критерий

![z_score_SE](z_score_SE.png "Z score SE")

- В числителе - разность выборочных конверсий (получившихся в результате эксперимента)
- В знаменателе - так называемая стандартная ошибка среднего (SE)

Посчитаем z-критерий "вручную"

In [4]:
import numpy as np


z_crit_value = 1.96 #соответствует доверительному интервалу в 95%
k1, n1 = data[data['ab_group']=='A']['purchase'].sum(), data[data['ab_group']=='A'].shape[0]
k2, n2 = data[data['ab_group']=='B']['purchase'].sum(), data[data['ab_group']=='B'].shape[0]
k1, k2, n1, n2

(256, 234, 11835, 11817)

Посчитаем выборочные p1, p2:

In [5]:
p1, p2 = k1/n1, k2/n2
p1, p2

(0.02163075623151669, 0.019801980198019802)

Посчитаем z-score:

![z_score_SE](z_score_SE.png "Z score SE")

In [6]:
P = (p1*n1+p2*n2)/(n1+n2)
z = (p1-p2)/(P*(1-P)*(1/n1+1/n2))**(1/2)
z

0.987293179904521

Сравним полученное значение с критическим, выбранным ранее

In [8]:
z_crit_value

1.96

In [9]:
if abs(z) > z_crit_value:
    print("We may reject the null hypothesis!")
else:
    print("We have failed to reject the null hypothesis")

We have failed to reject the null hypothesis


#### z-критерий, но уже воспользуемся возможностями библиотеки

In [10]:
from statsmodels.stats import proportion

z_score, z_pvalue = proportion.proportions_ztest(np.array([k1, k2]), 
                                                   np.array([n1, n2]))
print('Results are ','z_score =%.3f, pvalue = %.3f'%(z_score, z_pvalue))

Results are  z_score =0.987, pvalue = 0.323


#### давайте сразу же и непараметрический критерий посмотрим (хи-квадрат)

In [11]:
chisq, pvalue, table = proportion.proportions_chisquare(np.array([k1, k2]), 
                                                   np.array([n1, n2]))

print('Results are ','chisq =%.3f, pvalue = %.3f'%(chisq, pvalue))

Results are  chisq =0.975, pvalue = 0.323


### Что насчет других метрик?

Критерий мана-уитни для среднего чека (z-критерий уже не годится)

In [12]:
data.groupby('ab_group')['revenue'].describe()

,count,mean,std,min,25%,50%,75%,max
ab_group,,,,,,,,
A,11835.0,0.404462,13.133218,0.0,0.0,0.0,0.0,1303.609284
B,11817.0,0.244794,3.176534,0.0,0.0,0.0,0.0,113.830000


Мы видим, что вроде бы различия серьезные, но понимаем при этом, что все дело скорее всего в выбросах (а среднее у нас к ним не очень устойчиво)

In [13]:
from scipy.stats import mannwhitneyu

mw_stats = mannwhitneyu(x=data[data['ab_group'] == 'A']['revenue'].values,
                                                  y=data[data['ab_group'] == 'B']['revenue'].values)
mw_stats

MannwhitneyuResult(statistic=69799439.0, pvalue=0.16219447808712767)

Критерий Манна-Уитни нам <b>не позволяет</b> принять альтернативную гипотезу о разнице arpu

#### Может быть среднее количество покупок на пользователя отличается?

In [16]:
mw_stats = mannwhitneyu(x=data[data['ab_group'] == 'A']['num_purchases'].values,
                                                  y=data[data['ab_group'] == 'B']['num_purchases'].values)
mw_stats

MannwhitneyuResult(statistic=69799596.0, pvalue=0.16248592489460356)

#### Давайте еще посмотрим на arppu (average revenue per paying user)

In [17]:
mw_stats = mannwhitneyu(x=data[(data['ab_group'] == 'A')&(data['purchase'] == 1)]['revenue'].values,
                                                  y=data[(data['ab_group'] == 'B')&(data['purchase'] == 1)]['revenue'].values)
mw_stats

MannwhitneyuResult(statistic=29729.5, pvalue=0.4435978308172257)

In [18]:
data[data['purchase'] == 1].groupby('ab_group')['revenue'].describe()

,count,mean,std,min,25%,50%,75%,max
ab_group,,,,,,,,
A,256.0,18.698469,87.527589,0.259689,2.318301,4.28316,11.923141,1303.609284
B,234.0,12.362119,19.007065,0.259373,2.319883,3.98000,12.274829,113.830000


### Выводы:

1. ни по одной из метрик мы не видим значимых различий
2. скорее всего слишком мало данных для выводов (учитывая базовую конверсию в 2 процента)
3. мы также можем посчитать сколько нам понадобится приблизительно пользователей в каждой из групп, чтобы при такой конверсии получить значимость (для этого нам нужно делать предположение о том, какое минимальное изменение нас устраивает - минимально значимый эффект)**

** например, мы хотим, чтобы конверсия выросла хотя бы на 10% 

### Важные моменты

### 1. как может быть устроено разбиение выборки на группы

Часто для этого используется отдельный сервис, который умеет для выбранного user_id возвращать группу (причем в рамках одного теста одну и ту же)

Основные способы разбиения:

### Random

<img src="sampling_random.png" alt="random split" style="width: 600px;" align="center"/>

### Stratified random

<img src="sampling_stratified.png" alt="strat random split" style="width: 600px;" align="center"/>

Какой вариант предпочтительнее - ответ зависит от объемов аудитории преимущественно и того, насколько смещенной получается случайная выборка относительно генеральной совокупности

### 2. Контрольный сегмент (как оценить вклад ML-решения в целом)

Таким образом, у нас теперь есть представление, как сравнивать между собой разные варианты. 

Однако на практике также может возникнуть вопрос, а какой вклад ML в целом. Т.е если мы представим ситуацию, что никаких изменений на основе ML нет, так бы выглядел сервис в таком случае? (в плане бизнес-метрик)

Чтобы ответить на этот вопрос, иногда выделяют так называемый <b>контрольный сегмент</b> - пользователей, которых никогда не затрагивают изменения на базе машинного обучения.

Например, если мы внедряем систему рекомендаций, то пользователи из контрольного сегмента не увидят эти рекомендации. 

Обычно контрольный сегмент небольшой.

### 3. Когда вариантов несколько

1. поправка Бонферонни - https://en.wikipedia.org/wiki/Bonferroni_correction

Идея простая - для каждого попарного сравнения (пусть будет m таких сравнений) заменить alpha на alpha/m


Прекрасная ссылка для чтения на дом - https://changyaochen.github.io/multiple-comparisons-jun-2020/

### Домашнее задание

#### 1. Если мы рассчитали 95% доверительный интервал для среднего значения, то какие из следующих утверждений являются верными?

Выберите несколько вариантов и попытайтесь объяснить свой выбор:

1) Если многократно повторять эксперимент, то 95 % выборочных средних значений будут принадлежать рассчитанному нами доверительному интервалу.

2) Мы можем быть на 95% уверены, что среднее значение в генеральной совокупности принадлежит рассчитанному доверительному интервалу.

3) Если многократно повторять эксперимент, для каждой выборки рассчитывать свой доверительный интервал, то в 95 % случаев истинное среднее будет находиться внутри доверительного интервала.

4) Среднее значение в генеральной совокупности точно превышает нижнюю границу 95% доверительного интервала.

5) Среднее значение в генеральной совокупности точно принадлежит рассчитанному доверительному интервалу.

#### 2. Если мы увеличиваем объем выборки в два раза (при условии, что показатель стандартного отклонения остается неизменным), то 95% доверительный интервал

Выберите один вариант из списка

1) стал более узким
2) возможны оба варианта
3) стал более широким

#### 3. В центре 95% доверительного интервала, рассчитанного по выборочным значениям, находится:

Выберите один вариант из списка

1) Значение стандартной ошибки среднего
2) Выборочное среднее значение
3) Среднее значение генеральной совокупности

#### 4. Часто на практике нулевая гипотеза отклоняется, и различия считаются статистически достоверными, если p < 0,05. Однако часто в статистике используется более жесткий критерий достоверности различий, например, при условии, что p < 0,01. Значение p-уровня значимости, которое выбирается, в качестве порога обозначается буквой α (альфа). Например, если исследователь решил, что α = 0,05, то и нулевая гипотеза будет отклоняться при условии, что p < 0,05. 

#### Если в определенной ситуации весьма рискованно отклонить нулевую гипотезу, когда она на самом деле верна, то лучше использовать показатель α равный 

Выберите один вариант из списка

1) 0,1
2) 0,001
3) 0,05
4) 0,5

#### 5. Данные некоторого исследования сообщают нам, что средний рост детей в 14 лет составляет 166 сантиметров. Однако это лишь выборочная оценка, и исследователи рассчитали 95% доверительный интервал, который составил [160, 172]. Укажите верные утверждения:

Выберите несколько вариантов и попытайтесь объяснить свой выбор:

1) У нас достаточно оснований отклонить нулевую гипотезу, что среднее в генеральной совокупности равняется 173
2) Вероятность того, что истинное среднее значение больше 172, составляет 0,01
3) У нас достаточно оснований отклонить нулевую гипотезу, что среднее в генеральной совокупности равняется 158.
4) Доверительный интервал не может иметь такие границы, т. к. выборочное стандартное отклонение равняется 10, следовательно доверительный интервал должен быть значительно шире.

#### 6. Предположим, нулевой гипотезой вашего исследования являлось предположение, что конверсия в генеральной совокупности равняется 0.4. Вы получили p = 0,12 и не смогли отклонить нулевую гипотезу. Однако позже выяснилось, что конверсия в генеральной совокупности действиетльно равна 0.4. Как можно оценить результаты?


Какой вариант корректный по вашему мнению и почему?
1) Вы не совершали ни ошибку первого рода, ни ошибку второго рода.
2) Вы совершили ошибку первого рода
3) Вы совершили ошибку второго рода